In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
owid_data = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"

In [3]:
df = pd.read_csv(owid_data)

In [4]:
columns_ = ['date','location','iso_code','total_vaccinations','population']

## Herd Immunity
The Covid-19 basic reproduction number R0 is estimated at 2.5 [1] if no social measures are applied. To reach herd immunity we can apply the herd-immunity threshold with

p = 1 - (1/R0)


In [5]:
R0 = 2.5
p_raw = 1 - (1/R0)

print(f"p = {p_raw}")

p = 0.6


Meaning we need 60% of the population to be vaccinated to stop the virus’ spread.

However, we can assume the vaccine's effectiveness at 95% [2], and subtract the 4.4% [3] of the total population that is already immune after being infected.

    p = (1 - (1/R0) - 4.4%) / (95%)

In [6]:
IMUNE = 0.044
VACCINE_EFFICIENCY = 0.95

p = (1 - 1/R0 - IMUNE) / VACCINE_EFFICIENCY

print(f"p = {round(p,3)}")

p = 0.585


Therefore, the minimum number of individuals that require the vaccine to achieve herd immunity is around 58.5% of total population.

## Vaccination campaign

Each country has its own approach that depends on the manufacturer [4] and the administration methodology that we will not explore here.

Currently, by aggregating the doses administrated in Europe we can see the trend slowing down.

### Rolling average of vaccination

In [7]:
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511


In [8]:
df.date = pd.to_datetime(df.date)

In [9]:
df_pt = df[columns_].copy()
df_pt = df_pt[df_pt.location == 'Portugal']
df_pt = df_pt[df_pt.date > '2021-01-01'] # first date
df_pt.head()

,date,location,iso_code,total_vaccinations,population
65157,2021-01-02,Portugal,PRT,32750.0,10196707.0
65158,2021-01-03,Portugal,PRT,32750.0,10196707.0
65159,2021-01-04,Portugal,PRT,33670.0,10196707.0
65160,2021-01-05,Portugal,PRT,46433.0,10196707.0
65161,2021-01-06,Portugal,PRT,61454.0,10196707.0


In [10]:
gp_pt = df_pt.groupby('iso_code').apply(lambda group: group.interpolate(method='linear', 
                                                                     limit_direction='forward', axis=0))

gp_pt = gp_pt.groupby('date').sum().reset_index()
gp_pt['total_vaccinations_ma'] = gp_pt.total_vaccinations.rolling(7).mean()
gp_pt['daily_diff'] = gp_pt.total_vaccinations.diff()
gp_pt['daily_diff_ma'] = gp_pt.daily_diff.rolling(7).mean()

time_span = gp_pt.date.apply(lambda x: (x - gp_pt.date.min()).days) #days

gp_pt['due_date'] = (gp_pt.population * p) * time_span / (gp_pt.total_vaccinations / 2)

In [11]:
gp_pt.tail()

,date,total_vaccinations,population,total_vaccinations_ma,daily_diff,daily_diff_ma,due_date
130,2021-05-12,4216224.0,10196707.0,3.985300e+06,69311.0,84181.142857,368.010998
131,2021-05-13,4284623.0,10196707.0,4.066263e+06,68399.0,80962.571429,364.921795
132,2021-05-14,4357120.0,10196707.0,4.143150e+06,72497.0,76887.285714,361.589268
133,2021-05-15,4436962.0,10196707.0,4.216550e+06,79842.0,73400.428571,357.772581
134,2021-05-16,4566812.0,10196707.0,4.296717e+06,129850.0,80166.857143,350.213422


In [12]:
fig = make_subplots(1,1,specs=[[{"secondary_y": True}]])
fig.add_trace(go.Line(x=gp_pt["date"], y=gp_pt["daily_diff_ma"],
                     name="7 days moving average"),
             row = 1, col = 1,secondary_y=True)
fig.add_trace(go.Bar(x=gp_pt["date"], y=gp_pt["daily_diff"],
                    opacity=0.4, name='daily doses'),
             row = 1, col = 1,secondary_y=True)

fig.update_layout(title='Daily rythm of COVID-19 vaccination doses administered in Portugal')
fig.update_yaxes(title_text="Doses per day", secondary_y=True)
fig.show()

C:\Users\Etienne\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [13]:
fig = make_subplots(1,1,specs=[[{"secondary_y": True}]])
fig.add_trace(go.Line(x=gp_pt["date"], y=gp_pt["total_vaccinations"],
                     name="total vaccinations"),
             row = 1, col = 1,secondary_y=True)

fig.add_trace(go.Line(x=gp_pt["date"], y=gp_pt["population"],
                     name="population"),
             row = 1, col = 1,secondary_y=True)

fig.add_trace(go.Line(x=gp_pt["date"], y=gp_pt["population"]*p,
                     name="herd immunity"),
             row = 1, col = 1,secondary_y=True)

fig.update_layout(title='Total COVID-19 vaccination doses administered in Portugal')
fig.update_yaxes(title_text="Total doses", secondary_y=True)
fig.show()

C:\Users\Etienne\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [14]:
gp_pt.tail()

,date,total_vaccinations,population,total_vaccinations_ma,daily_diff,daily_diff_ma,due_date
130,2021-05-12,4216224.0,10196707.0,3.985300e+06,69311.0,84181.142857,368.010998
131,2021-05-13,4284623.0,10196707.0,4.066263e+06,68399.0,80962.571429,364.921795
132,2021-05-14,4357120.0,10196707.0,4.143150e+06,72497.0,76887.285714,361.589268
133,2021-05-15,4436962.0,10196707.0,4.216550e+06,79842.0,73400.428571,357.772581
134,2021-05-16,4566812.0,10196707.0,4.296717e+06,129850.0,80166.857143,350.213422
